In [3]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor

In [4]:
df=pd.read_csv('./test/TV.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_Name      666 non-null    object 
 1   Stars             666 non-null    float64
 2   Ratings           666 non-null    int64  
 3   Reviews           666 non-null    int64  
 4   current_price     666 non-null    int64  
 5   MRP               666 non-null    int64  
 6   channel           666 non-null    object 
 7   Operating_system  666 non-null    object 
 8   Picture_quality   666 non-null    object 
 9   Speaker           666 non-null    object 
 10  Frequency         666 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 57.4+ KB


# 문제 풀기: ~ 4:10 

# Q1

In [5]:
df.Frequency.value_counts()

Frequency
60 Hz Refresh Rate                                                                                     423
50 Hz Refresh Rate                                                                                      53
100 Hz Refresh Rate                                                                                     25
120 Hz Refresh Rate                                                                                     23
200 Hz Refresh Rate                                                                                     14
3 x HDMI | 2 x USB                                                                                      13
A+ Grade                                                                                                12
A+                                                                                                      12
1 Year Warranty                                                                                          6
A+ Grade Panel             

In [11]:
# df.columns

In [7]:
var_list1=['Frequency','Picture_quality', 'Speaker']

In [10]:
df[var_list1].apply(lambda x: x.str.contains('60 Hz')).any(axis=1).sum()

510

# Q2

In [12]:
df.columns

Index(['Product_Name', 'Stars', 'Ratings', 'Reviews', 'current_price', 'MRP',
       'channel', 'Operating_system', 'Picture_quality', 'Speaker',
       'Frequency'],
      dtype='object')

In [13]:
var_list2=['Operating_system', 'channel', 'Picture_quality']

In [15]:
q2_8k=df[var_list2].apply(lambda x: x.str.contains('8K')).any(axis=1)
q2_4k=df[var_list2].apply(lambda x: x.str.contains('4K')).any(axis=1)

In [17]:
round(abs(df[q2_8k]['Stars'].mean() - df[q2_4k]['Stars'].mean()),2)

0.38

# Q3

In [20]:
df.channel.unique()

array(['HD Ready 1366 x 768 Pixels', 'Netflix|Disney+Hotstar|Youtube',
       'Netflix|Prime Video|Disney+Hotstar|Youtube',
       'HD Ready 1366 X 768 Pixels', 'Netflix|Prime Video|Youtube',
       'Youtube', 'Prime Video|Disney+Hotstar|Youtube',
       'Prime Video|Youtube', 'Netflix|Youtube',
       'HD Ready 1366 x 786 Pixels', 'Disney+Hotstar|Youtube',
       'HD Ready 1366*768 Pixels', 'Ultra HD (4K) 3840 x 2160 Pixels',
       'Netflix|Prime Video|Apple TV|Disney+Hotstar|Youtube',
       'HD Ready 720p Pixels', 'Ultra HD (4K) ?3840 x 2160 Pixels',
       'Operating System: Linux', 'Full HD 1920 x 1080 Pixels',
       'Operating System: Android', 'Ultra HD (4K) 3840*2160 Pixels',
       'Operating System: WebOS', 'Operating System: Tizen',
       'Ultra HD (4K) 3840 x 2160 - Ultra HD Pixels',
       'Ultra HD (4K) 4K QLED (3840 x 2160) Pixels', 'Prime Video',
       'Ultra HD (4K) 3840 * 2160 Pixels', 'HD Ready 1366X768P Pixels',
       'Full HD 1980 x 1080 Pixels', 'Operating Sy

In [18]:
q3=df.copy()
q3=q3[~q3.channel.str.contains('Pixel|Oper', regex=True)]

In [19]:
len(df), len(q3)

(666, 548)

In [21]:
q3['후기작성비율']=q3['Reviews']/q3['Ratings']
q3['공장출고가']=q3['MRP'].copy()
q3['할인율']=q3['current_price'] / q3['공장출고가']
q3['Netflix_yn']=q3.channel.str.contains('Netflix') + 0
q3['Prime_Video_yn']=q3.channel.str.contains('Prime Video') + 0
q3['고해상도여부']=q3.Picture_quality.str.contains('4K|8K', regex=True) + 0

In [22]:
# q3.Picture_quality.str.contains('4K|8K') + 0

1      0
2      0
3      0
4      0
5      0
      ..
655    0
656    1
657    0
662    1
664    0
Name: Picture_quality, Length: 548, dtype: int32

In [25]:
var_list3=q3.columns[-6:]

In [27]:
q3[var_list3].isna().sum()

후기작성비율            351
공장출고가               0
할인율                 0
Netflix_yn          0
Prime_Video_yn      0
고해상도여부              0
dtype: int64

In [28]:
q3_f=q3.dropna(subset=var_list3)

In [29]:
len(q3_f)

197

In [30]:
rf=RandomForestRegressor(random_state=123).fit(q3_f[var_list3], q3_f['Stars'])

In [33]:
pd.Series(rf.feature_importances_, index=var_list3).idxmax()

'할인율'